#### Extract all AcinoSet data and merge all .csv files:

In [ ]:
import os
import pandas as pd
from PIL import Image

# NB: unzip "labelled_data.zip" to "/notebooks/data/acino/labelled" first

acino='/notebooks/data/acino'

img_ids=[os.path.splitext(img)[0] for img in os.listdir(acino+'/../ap10k/data/')]
img_ids.remove('.ipynb_checkpoints')
img_ids=[int(i) for i in img_ids]
img_no=max(img_ids)+1

base=pd.read_csv('base.csv')

for dir in os.listdir(acino+'/labelled'): #JamesFlick1 JamesFlick2 etc.
    print(dir)
    if os.path.isdir(acino+'/labelled/'+dir):
        
        df=pd.read_csv('labelled/'+dir+'/'+"CollectedData_UCT.csv")

        df.iloc[0,1:]=df.iloc[0,1:].astype(str) +'_'+ df.iloc[1,1:].astype(str)
        df.columns=df.iloc[0] # Make keypoint labels column headers
        df=df.iloc[2:,:]
        
        #Extract COCO keypoints (AP10K):
        #df=df.reindex(columns=(['bodyparts','l_eye_x','l_eye_y','r_eye_x','r_eye_y','nose_x','nose_y','neck_base_x','neck_base_y','tail_base_x','tail_base_y','l_shoulder_x','l_shoulder_y','l_front_knee_x','l_front_knee_y','l_front_paw_x','l_front_paw_y','r_shoulder_x','r_shoulder_y','r_front_knee_x','r_front_knee_y','r_front_paw_x','r_front_paw_y','l_hip_x','l_hip_y','l_back_knee_x','l_back_knee_y','l_back_paw_x','l_back_paw_y','r_hip_x','r_hip_y','r_back_knee_x','r_back_knee_y','r_back_paw_x','r_back_paw_y','tail2_x','tail2_y']))
        
        #Extract COCO keypoints (All Acino Keypoints):
        df=df.reindex(columns=(['bodyparts','l_eye_x','l_eye_y','r_eye_x','r_eye_y','nose_x','nose_y','neck_base_x','neck_base_y','tail_base_x','tail_base_y','l_shoulder_x','l_shoulder_y','l_front_knee_x','l_front_knee_y','l_front_paw_x','l_front_paw_y','r_shoulder_x','r_shoulder_y','r_front_knee_x','r_front_knee_y','r_front_paw_x','r_front_paw_y','l_hip_x','l_hip_y','l_back_knee_x','l_back_knee_y','l_back_paw_x','l_back_paw_y','r_hip_x','r_hip_y','r_back_knee_x','r_back_knee_y','r_back_paw_x','r_back_paw_y','tail2_x','tail2_y','tail1_x','tail1_y','r_front_ankle_x','r_front_ankle_y','l_front_ankle_x','l_front_ankle_y','r_back_ankle_x','r_back_ankle_y','l_back_ankle_x','l_back_ankle_y','spine_x','spine_y']))
        
        for i in range(0,len(df)): # Iterate over rows
            orig=df.iloc[i,0].split('/')[2]
            df.iloc[i,0]='as_'+str(img_no)+'.jpg'
            
            im=Image.open('data/acino/labelled/'+dir+'/'+orig)
            img_name='as_'+str(img_no)
            im.save('data/acino/data/'+img_name+'.jpg')
            
            #os.remove('labelled1/'+dir+'/'+orig)
            
            
            img_no+=1
        base=pd.concat([base,df[0:]],ignore_index=True)
               
base.to_csv("data/acino/annotations.csv",index=False)

#### Convert .csv labels to COCO .json format:

In [1]:
import os
import pandas as pd
import random
import json
import numpy as np
import cv2

df = pd.read_csv('data/acino/annotations.csv')

with open('data/acino/annotations/acino_all.json','r+') as f: # Save all annotation instances in one .json file
    anns_data=json.load(f)
    
    for i in range(0,len(df)):# Iterate over each image

        #Add image instance
        img_name=df.iloc[i,0]
        img_id=int(img_name[3:].split('.')[0])
        img=cv2.imread('data/acino/data/'+img_name)
        height = img.shape[0]
        width = img.shape[1]
        img_inst=dict(width=width,height=height,file_name=img_name,background=1,id=img_id)
        
        #Add annotation instance
        ann_id=img_id
        kp=[]
        num_kp=0

        # Initial bbox:
        x_min=width-1
        x_max=1
        y_min=height-1
        y_max=1

        for j in range(1,len(df.columns),2):
            if pd.isnull(df.iloc[i,j]): #NaN values
                kp.append(0)
                kp.append(0)
                kp.append(0)
                
            else:
                x=int(round(pd.to_numeric(df.iloc[i,j])))
                y=int(round(pd.to_numeric(df.iloc[i,j+1])))
                
                #Add annotations
                kp.append(x)
                kp.append(y)
                kp.append(2)
                num_kp+=1

                #Bbox
                if x<x_min:
                    x_min=x
                if x>x_max:
                    x_max=x
                if y>y_max:
                    y_max=y
                if y<y_min:
                    y_min=y

        w=max(1,(x_max-x_min)) # BBox width
        h=max(1,(y_max-y_min)) # BBox height
        a=w*h # Bbox area
        
        #Padded bounding box (10 pixels all sides):
        bbox=[x_min-10,y_min-10,w+20,h+20] # xywh format NB! y_min==top left corner
        
#         if not kp[-1]==0: # If tip of tail != NaN
#             num_kp-=1 
            
#         del kp[-3:] # Remove tail_x, tail_y, visible
        
        
        ann_inst=dict(image_id=img_id,iscrowd=0,category_id=25,num_keypoints=num_kp,keypoints=kp,bbox=bbox,id=ann_id,area=a)
        anns_data['images'].append(img_inst)
        anns_data['annotations'].append(ann_inst)
    f.seek(0)
    json.dump(anns_data,f,indent=4)
f.close()

#### Train-test-val split:

In [2]:
%pip install echo1-coco-split

Note: you may need to restart the kernel to use updated packages.


In [3]:
!coco-split --annotations_file data/acino/annotations/acino_all.json --valid_ratio 0.1 --test_ratio 0.2 --train_name data/acino/annotations/acino_train.json --valid_name data/acino/annotations/acino_val.json --test_name data/acino/annotations/acino_test.json --has_annotations

Saved 5346 annotations from 5346 images to data/acino/annotations/acino_train.json
Saved 764 annotations from 764 images to data/acino/annotations/acino_val.json
Saved 1528 annotations from 1528 images to data/acino/annotations/acino_test.json


#### Train Model:

In [4]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
%pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
%pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

# install mmdet for inference demo
%pip install mmdet

# install mmpose dependencies
%pip install -r requirements.txt

# install mmpose in develop mode
%pip install -e .

# Check Pytorch installation
import torch, torchvision

# print('torch version:', torch.__version__, torch.cuda.is_available())
# print('torchvision version:', torchvision.__version__)

# # Check MMPose installation
# import mmpose

# print('mmpose version:', mmpose.__version__)

# Check mmcv installation
# from mmcv.ops import get_compiling_cuda_version, get_compiler_version

# print('cuda version:', get_compiling_cuda_version())
# print('compiler information:', get_compiler_version())

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 GB 884.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 31.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu116
    Uninstalling torch-1.12.0+cu116:
      Successfully uninstalled torch-1.12.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cu116
    Uninstalling torchvision-0.13.0+cu116:
      Successfully uninstalled torchvision-0.13.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.12.0+cu116 requires torch==1.12.0, but you have torch 1.10.0+cu111 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Looking in links: https://download.openmmlab

In [7]:
# Config file:
from mmcv import Config
import mmcv
import mmpose
import torch, torchvision

cfg = Config.fromfile('configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/hrnet_w32_acino_256x256.py'
    #'./configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w32_coco_256x192.py'
)
# cfg.data_root = 'data'
# cfg.work_dir = 'work_dirs/hrnet_w32_acino_256x256'
# cfg.data['train']['ann_file']=f'{cfg.data_root}/acino/annotations/acino_train.json'
# cfg.data['train']['img_prefix']=f'{cfg.data_root}/acino/data/'
# cfg.data['test']['ann_file']=f'{cfg.data_root}/acino/annotations/acino_test.json'
# cfg.data['test']['img_prefix']=f'{cfg.data_root}/acino/data/'
# cfg.data['val']['ann_file']=f'{cfg.data_root}/acino/annotations/acino_val.json'
# cfg.data['val']['img_prefix']=f'{cfg.data_root}/acino/data/'
# cfg.gpu_ids = range(1)
# cfg.seed = 0
cfg.total_epochs=50

In [13]:
# Train from config file
from mmpose.datasets import build_dataset
from mmpose.models import build_posenet
from mmpose.apis import train_model

# build dataset
datasets = [build_dataset(cfg.data.train)]

# build model
model = build_posenet(cfg.model)

# create work_dir
mmcv.mkdir_or_exist(cfg.work_dir)

# train model
train_model(
    model, datasets, cfg, distributed=False, validate=True, meta=dict())

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
=> num_images: 5346
=> load 5289 samples
load checkpoint from http path: https://download.openmmlab.com/mmpose/pretrain_models/hrnet_w32-36af842e.pth


2022-09-12 07:23:50,075 - mmpose - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.0.0.0.conv1.weight, head.0.0.0.bn1.weight, head.0.0.0.bn1.bias, head.0.0.0.bn1.running_mean, head.0.0.0.bn1.running_var, head.0.0.0.bn1.num_batches_tracked, head.0.0.0.conv2.weight, head.0.0.0.bn2.weight, head.0.0.0.bn2.bias, head.0.0.0.bn2.running_mean, head.0.0.0.bn2.running_var, head.0.0.0.bn2.num_batches_tracked, head.0.0.0.conv3.weight, head.0.0.0.bn3.weight, head.0.0.0.bn3.bias, head.0.0.0.bn3.running_mean, head.0.0.0.bn3.running_var, head.0.0.0.bn3.num_batches_tracked, head.0.0.0.downsample.0.weight, head.0.0.0.downsample.1.weight, head.0.0.0.downsample.1.bias, head.0.0.0.downsample.1.running_mean, head.0.0.0.downsample.1.running_var, head.0.0.0.downsample.1.num_batches_tracked, head.0.1.0.conv1.weight, head.0.1.0.bn1.weight, head.0.1.0.bn1.bias, head.0.1.0.bn1.running_mean, head.0.1.0.bn1.running_var, head.0.1.0.bn1.num_batches_tracked, he

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
=> num_images: 764
=> load 757 samples


2022-09-12 07:23:57,846 - mmpose - INFO - Epoch [1][1/83]	lr: 5.000e-07, eta: 8:36:21, time: 7.467, data_time: 5.973, memory: 12606, heatmap_loss: 0.0015, acc_pose: 0.0163, loss: 0.0015
2022-09-12 07:23:59,044 - mmpose - INFO - Epoch [1][2/83]	lr: 1.499e-06, eta: 4:59:31, time: 1.198, data_time: 0.006, memory: 12939, heatmap_loss: 0.0016, acc_pose: 0.0300, loss: 0.0016
2022-09-12 07:24:00,235 - mmpose - INFO - Epoch [1][3/83]	lr: 2.498e-06, eta: 3:47:04, time: 1.191, data_time: 0.003, memory: 12939, heatmap_loss: 0.0016, acc_pose: 0.0255, loss: 0.0016
2022-09-12 07:24:01,419 - mmpose - INFO - Epoch [1][4/83]	lr: 3.497e-06, eta: 3:10:42, time: 1.183, data_time: 0.006, memory: 12939, heatmap_loss: 0.0015, acc_pose: 0.0180, loss: 0.0015
2022-09-12 07:24:02,572 - mmpose - INFO - Epoch [1][5/83]	lr: 4.496e-06, eta: 2:48:28, time: 1.154, data_time: 0.008, memory: 12939, heatmap_loss: 0.0017, acc_pose: 0.0247, loss: 0.0017
2022-09-12 07:24:03,718 - mmpose - INFO - Epoch [1][6/83]	lr: 5.495e-0

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 757/757, 63.9 task/s, elapsed: 12s, ETA:     0sLoading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=0.83s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.735
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.980
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.875
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.510
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.775
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.775
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.987
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.892
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium 

...

2022-09-12 08:32:51,444 - mmpose - INFO - The previous best checkpoint /notebooks/work_dirs/hrnet_w32_acino_256x256/best_AP_epoch_30.pth was removed


DONE (t=0.84s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.850
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.990
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.930
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.645
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.886
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.877
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.992
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.945
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.692
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.914


2022-09-12 08:32:52,618 - mmpose - INFO - Now best checkpoint is saved as best_AP_epoch_40.pth.
2022-09-12 08:32:52,619 - mmpose - INFO - Best AP is 0.8501 at 40 epoch.
2022-09-12 08:32:52,620 - mmpose - INFO - Epoch(val) [40][24]	AP: 0.8501, AP .5: 0.9900, AP .75: 0.9304, AP (M): 0.6449, AP (L): 0.8856, AR: 0.8771, AR .5: 0.9921, AR .75: 0.9445, AR (M): 0.6917, AR (L): 0.9135
2022-09-12 08:32:59,343 - mmpose - INFO - Epoch [41][1/83]	lr: 5.000e-04, eta: 0:16:58, time: 6.715, data_time: 5.538, memory: 12939, heatmap_loss: 0.0004, acc_pose: 0.9048, loss: 0.0004
2022-09-12 08:33:00,520 - mmpose - INFO - Epoch [41][2/83]	lr: 5.000e-04, eta: 0:16:57, time: 1.178, data_time: 0.007, memory: 12939, heatmap_loss: 0.0004, acc_pose: 0.8766, loss: 0.0004
2022-09-12 08:33:01,720 - mmpose - INFO - Epoch [41][3/83]	lr: 5.000e-04, eta: 0:16:56, time: 1.200, data_time: 0.003, memory: 12939, heatmap_loss: 0.0004, acc_pose: 0.8752, loss: 0.0004
2022-09-12 08:33:02,882 - mmpose - INFO - Epoch [41][4/83]	

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 757/757, 64.8 task/s, elapsed: 12s, ETA:     0sLoading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=0.80s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.858
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.990
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.932
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.677
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.892
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.885
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.993
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.945
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium 

2022-09-12 08:50:06,076 - mmpose - INFO - The previous best checkpoint /notebooks/work_dirs/hrnet_w32_acino_256x256/best_AP_epoch_40.pth was removed
2022-09-12 08:50:07,422 - mmpose - INFO - Now best checkpoint is saved as best_AP_epoch_50.pth.
2022-09-12 08:50:07,423 - mmpose - INFO - Best AP is 0.8578 at 50 epoch.
2022-09-12 08:50:07,424 - mmpose - INFO - Epoch(val) [50][24]	AP: 0.8578, AP .5: 0.9901, AP .75: 0.9320, AP (M): 0.6768, AP (L): 0.8924, AR: 0.8851, AR .5: 0.9934, AR .75: 0.9445, AR (M): 0.7150, AR (L): 0.9186


In [14]:
# Test model:
from matplotlib import pprint
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results)
# from mmdet.apis import inference_detector, init_detector
from xtcocotools.coco import COCO
import os
# import cv2
local_runtime = False

# try:
#     from google.colab.patches import cv2_imshow  # for image visualization in colab
# except:
#     local_runtime = True
from mmpose.datasets import DatasetInfo

pose_checkpoint = 'work_dirs/hrnet_w32_acino_256x256/latest.pth'
json_file='tests/data/acino/testacino.json'
coco = COCO(json_file)
img_keys = list(coco.imgs.keys())

# initialize pose model
pose_model = init_pose_model(cfg, pose_checkpoint)

dataset = pose_model.cfg.data['test']['type']
dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
dataset_info = DatasetInfo(dataset_info)
return_heatmap=False
output_layer_names=None

for i in mmcv.track_iter_progress(range(len(img_keys))):
  # get bounding box annotations
  image_id = img_keys[i]
  #print("\nID:",image_id,"\n")
  image = coco.loadImgs(image_id)[0]
  #print("\nID:",image,"\n")
  image_name = os.path.join('tests/data/acino', image['file_name'])
  #print("\nID:",image_name,"\n")
  ann_ids = coco.getAnnIds(image_id)
  
  person_results = []
  for ann_id in ann_ids:
    person = {}
    ann = coco.anns[ann_id]
    
    # bbox format is 'xywh'
    person['bbox'] = ann['bbox']
    person_results.append(person)
    # print('\n')
    # print(person_results)
    # print('\n')

  pose_results, returned_outputs = inference_top_down_pose_model(
    pose_model,
    image_name,
    person_results=person_results,##
    bbox_thr=None,
    format='xywh',
    dataset=dataset,
    dataset_info=dataset_info,
    return_heatmap=return_heatmap,
    outputs=output_layer_names)
  
  os.makedirs('vis_result', exist_ok=True)
  out_file = os.path.join('vis_result', f'vis_as{image_id}.jpg')
  vis_result=vis_pose_result(pose_model,image_name,pose_results,radius=3,thickness=1,out_file=out_file)
  

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: work_dirs/hrnet_w32_acino_256x256/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2/2, 1.9 task/s, elapsed: 1s, ETA:     0s
